# RAG: Fundamentos e técnicas avançadas

 > Aviso: Este post foi traduzido para o português usando um modelo de tradução automática. Por favor, me avise se encontrar algum erro.

Neste post vamos a ver em que consiste a técnica de `RAG` (`Retrieval Augmented Generation`) e como se pode implementar em um modelo de linguagem.

Para que saia de graça, em vez de usar uma conta do OpenAI (como você verá na maioria dos tutoriais) vamos usar o `API inference` do Hugging Face, que tem um plano gratuito de 1000 requisições por dia, que para fazer este post é mais do que suficiente.

## Configuração da `API Inference` da Hugging Face

Para poder usar a `API Inference` da HuggingFace, o primeiro que você precisa é ter uma conta na HuggingFace, uma vez que a tenha, é necessário ir para [Access tokens](https://huggingface.co/settings/keys) nas configurações do seu perfil e gerar um novo token.

Temos que colocar um nome, no meu caso vou colocá-lo como `rag-fundamentals` e habilitar a permissão `Make calls to serverless Inference API`. Será criado um token que temos que copiar

Para gerenciar o token vamos a criar um arquivo na mesma rota em que estivermos trabalhando chamado `.env` e vamos a colocar o token que copiamos no arquivo da seguinte maneira:

``` bash
RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN="hf_...."
```

Agora para poder obter o token precisamos ter instalado `dotenv`, que o instalamos mediante

```bash
pip install python-dotenv
```

E executamos o seguinte

In [1]:
import os
import dotenv

dotenv.load_dotenv()

RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN = os.getenv("RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN")

Agora que temos um token, criamos um cliente, para isso precisamos ter instalada a biblioteca `huggingface_hub`, que fazemos mediante conda ou pip

``` bash
conda install -c conda-forge huggingface_hub
```

o

``` bash
pip install --upgrade huggingface_hub
```

Agora temos que escolher qual modelo vamos usar. Você pode ver os modelos disponíveis na página de [Supported models](https://huggingface.co/docs/api-inference/supported-models) da documentação da `API Inference` da Hugging Face.

Como na hora de escrever o post, o melhor disponível é `Qwen2.5-72B-Instruct`, vamos usar esse modelo.

In [2]:
MODEL = "Qwen/Qwen2.5-72B-Instruct"

Agora podemos criar o cliente

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN, model=MODEL)
client

<InferenceClient(model='Qwen/Qwen2.5-72B-Instruct', timeout=None)>

Fazemos uma prova para ver se funciona

In [4]:
message = [
	{ "role": "user", "content": "Hola, qué tal?" }
]

stream = client.chat.completions.create(
	messages=message, 
	temperature=0.5,
	max_tokens=1024,
	top_p=0.7,
	stream=False
)

response = stream.choices[0].message.content
print(response)

¡Hola! Estoy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?


## O que é `RAG`?

`RAG` são as siglas de `Retrieval Augmented Generation`, é uma técnica criada para obter informações de documentos. Embora os LLMs possam chegar a ser muito poderosos e ter muito conhecimento, nunca vão ser capazes de responder sobre alguns documentos privados, como relatórios da sua empresa, documentação interna, etc. Por isso foi criado o `RAG`, para poder usar esses LLMs nessa documentação privada.

![O que é RAG?](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/RAG.webp)

A ideia consiste em que um usuário faz uma pergunta sobre essa documentação privada, o sistema é capaz de obter a parte da documentação na qual está a resposta para essa pergunta, passa-se a pergunta e a parte da documentação para um LLM e o LLM gera a resposta para o usuário

### Como a informação é armazenada?

É sabido, e se não sabia, agora estou te contando, que os LLMs têm um limite de informação que pode ser passada para eles, isso é chamado de janela de contexto. Isso ocorre devido às arquiteturas internas dos LLMs, que agora não vêm ao caso. Mas o importante é que não se pode passar um documento e uma pergunta sem mais, porque é provável que o LLM não seja capaz de processar toda essa informação.

Nos casos em que se costuma passar mais informações do que a janela de contexto permite, o que geralmente acontece é que o LLM não presta atenção ao final da entrada. Imagine que você pergunte ao LLM algo sobre o seu documento, que essa informação esteja no final do documento e o LLM não a leia.

Por isso, o que se faz é dividir a documentação em blocos chamados `chunk`s. De modo que a documentação é armazenada em um monte de `chunk`s, que são pedaços daquela documentação. Assim, quando o usuário faz uma pergunta, passa-se ao LLM o `chunk` em que está a resposta para aquela pergunta.

Além de dividir a documentação em `chunk`s, esses são convertidos em embeddings, que são representações numéricas dos `chunk`s. Isso é feito porque os LLMs na verdade não entendem texto, mas números, e os `chunk`s são convertidos em números para que o LLM possa entendê-los. Se você quiser entender mais sobre os embeddings, pode ler meu post sobre [transformers](https://www.maximofn.com/transformers) no qual explico como funcionam os transformers, que é a arquitetura por trás dos LLMs. Você também pode ler meu post sobre [ChromaDB](https://www.maximofn.com/chromadb) onde explico como os embeddings são armazenados em um banco de dados vetorial. E além disso, seria interessante que lesse meu post sobre a biblioteca [HuggingFace Tokenizers](https://www.maximofn.com/hugging-face-tokenizers) na qual se explica como o texto é tokenizado, que é o passo anterior à geração dos embeddings

![RAG - embeddings](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/RAG-embeddings.webp)

### Como se obtém o `chunk` correto?

Dissemos que a documentação se divide em `chunk`s e se passa ao LLM o `chunk` no qual está a resposta à pergunta do usuário. Mas, como se sabe em qual `chunk` está a resposta? Para isso, o que se faz é converter a pergunta do usuário em um embedding, e se calcula a similaridade entre o embedding da pergunta e os embeddings dos `chunk`s. De modo que o `chunk` com maior similaridade é o que se passa ao LLM.

![RAG - semelhança de embeddings](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/rag-chunk_retreival.webp)

### Vamos rever o que é `RAG`

Por um lado, temos o `retrieval`, que é obter o `chunk` correto da documentação, por outro lado, temos o `augmented`, que é passar ao LLM a pergunta do usuário e o `chunk` e, por último, temos o `generation`, que é obter a resposta gerada pelo LLM.

## Base de dados vetorial

Vimos que a documentação se divide em `chunk`s e é salva em um banco de dados vetorial, por isso precisamos usar um. Para este post vou usar [ChromaDB](https://www.trychroma.com/), que é um banco de dados vetorial bastante usado e sobre o qual também tenho um [post](https://www.maximofn.com/chromadb) em que explico como funciona.

Para isso, primeiro precisamos instalar a biblioteca do ChromaDB, para isso a instalamos com Conda ou com pip

``` bash
conda install conda-forge::chromadb
```

o

``` bash
pip install chromadb
```

### Função de embedding

Como dissemos, tudo se baseará em embeddings, por isso o primeiro que fazemos é criar uma função para obter embeddings de um texto. Vamos usar o modelo `sentence-transformers/all-MiniLM-L6-v2`

In [5]:
import chromadb.utils.embedding_functions as embedding_functions

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
      
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=RAG_FUNDAMENTALS_ADVANCE_TECHNIQUES_TOKEN,
    model_name=EMBEDDING_MODEL
)

Testamos a função de embedding

In [6]:
embedding = huggingface_ef(["Hello, how are you?",])
embedding[0].shape

(384,)

Obtemos um embedding de dimensão 384. Embora a missão deste post não seja explicar os embeddings, em resumo, nossa função de embedding categorizou a frase `Hello, how are you?` em um espaço de 384 dimensões.

### Cliente ChromaDB

Agora que temos nossa função de embedding podemos criar um cliente de ChromaDB

Primeiro criamos uma pasta onde será guardado o banco de dados vetorial

In [7]:
from pathlib import Path
      
chroma_path = Path("chromadb_persisten_storage")
chroma_path.mkdir(exist_ok=True)

Agora criamos o cliente

In [8]:
from chromadb import PersistentClient

chroma_client = PersistentClient(path = str(chroma_path))

### Coleção

Quando temos o cliente do ChromaDB, o seguinte que precisamos é criar uma coleção. Uma coleção é um conjunto de vetores, no nosso caso os `chunks` da documentação.

Criamos indicando-lhe a função de embedding que vamos a usar

In [9]:
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(name=collection_name, embedding_function=huggingface_ef)

## Carga de documentos

Agora que criamos a base de dados vetorial, temos que dividir a documentação em `chunk`s e salvá-los na base de dados vetorial.

### Função de carregamento de documentos

Primeiro criamos uma função para carregar todos os documentos `.txt` de um diretório

In [10]:
def load_one_document_from_directory(directory, file):
    with open(os.path.join(directory, file), "r") as f:
        return {"id": file, "text": f.read()}

def load_documents_from_directory(directory):
    documents = []
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            documents.append(load_one_document_from_directory(directory, file))
    return documents


### Função para dividir a documentação em `chunk`s

Uma vez que temos os documentos, os dividimos em `chunk`s

In [11]:
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks


### Função para gerar embeddings de um `chunk`

Agora que temos os `chunk`s, geramos os `embedding`s de cada um deles

Depois veremos por quê, mas para gerar os embeddings vamos a fazê-lo de maneira local e não por meio da API da Hugging Face. Para isso, precisamos ter instalado o [PyTorch](https://pytorch.org) e `sentence-transformers`, para isso fazemos

``` bash
pip install -U sentence-transformers
```

In [12]:
from sentence_transformers import SentenceTransformer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_model = SentenceTransformer(EMBEDDING_MODEL).to(device)

def get_embeddings(text):
    try:
        embedding = embedding_model.encode(text, device=device)
        return embedding
    except Exception as e:
        print(f"Error: {e}")
        exit(1)

Vamos a testar agora esta função de embeddings localmente

In [13]:
text = "Hello, how are you?"
embedding = get_embeddings(text)
embedding.shape

(384,)

Vemos que obtemos um embedding da mesma dimensão que quando o fazíamos com a API da Hugging Face

O modelo `sentence-transformers/all-MiniLM-L6-v2` tem apenas 22M de parâmetros, por isso você poderá executá-lo em qualquer GPU. Mesmo que você não tenha uma GPU, poderá executá-lo em uma CPU.

O LLM que vamos a usar para gerar as respostas, que é o `Qwen2.5-72B-Instruct`, como o seu nome indica, é um modelo de 72B de parâmetros, por isso este modelo não pode ser executado em qualquer GPU e em uma CPU é impensável devido à lentidão. Por isso, este LLM sim o usaremos por meio da API, mas na hora de gerar os `embedding`s podemos fazer localmente sem problemas

### Documentos com os quais vamos testar

Para fazer todas estas provas, eu baixei o conjunto de dados [aws-case-studies-and-blogs](https://www.kaggle.com/datasets/harshsinghal/aws-case-studies-and-blogs) e o coloquei na pasta `rag-txt_dataset`, com os seguintes comandos eu digo como baixá-lo e descomprimi-lo

Criamos a pasta onde vamos baixar os documentos

In [30]:
!mkdir rag_txt_dataset

Descarregamos o `.zip` com os documentos

In [1]:
!curl -L -o ./rag_txt_dataset/archive.zip https://www.kaggle.com/api/v1/datasets/download/harshsinghal/aws-case-studies-and-blogs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1430k  100 1430k    0     0  1082k      0  0:00:01  0:00:01 --:--:-- 2440k


Descomprimimos o `.zip`

In [2]:
!unzip rag_txt_dataset/archive.zip -d rag_txt_dataset

Archive:  rag_txt_dataset/archive.zip
  inflating: rag_txt_dataset/23andMe Case Study _ Life Sciences _ AWS.txt  
  inflating: rag_txt_dataset/36 new or updated datasets on the Registry of Open Data_ AI analysis-ready datasets and more _ AWS Public Sector Blog.txt  
  inflating: rag_txt_dataset/54gene _ Case Study _ AWS.txt  
  inflating: rag_txt_dataset/6sense Case Study.txt  
  inflating: rag_txt_dataset/ADP Developed an Innovative and Secure Digital Wallet in a Few Months Using AWS Services _ Case Study _ AWS.txt  
  inflating: rag_txt_dataset/AEON Case Study.txt  
  inflating: rag_txt_dataset/ALTBalaji _ Amazon Web Services.txt  
  inflating: rag_txt_dataset/AWS Case Study - Ineos Team UK.txt  
  inflating: rag_txt_dataset/AWS Case Study - StreamAMG.txt  
  inflating: rag_txt_dataset/AWS Case Study_ Creditsafe.txt  
  inflating: rag_txt_dataset/AWS Case Study_ Immowelt.txt  
  inflating: rag_txt_dataset/AWS Customer Case Study _ Kepler Provides Effective Monitoring of Elderly Care 

Removamos o `.zip`

In [3]:
!rm rag_txt_dataset/archive.zip

Vemos o que nos quedou

In [4]:
!ls rag_txt_dataset

'23andMe Case Study _ Life Sciences _ AWS.txt'
'36 new or updated datasets on the Registry of Open Data_ AI analysis-ready datasets and more _ AWS Public Sector Blog.txt'
'54gene _ Case Study _ AWS.txt'
'6sense Case Study.txt'
'Accelerate Time to Business Value Using Amazon SageMaker at Scale with NatWest Group _ Case Study _ AWS.txt'
'Accelerate Your Analytics Journey on AWS with DXC Analytics and AI Platform _ AWS Partner Network (APN) Blog.txt'
'Accelerating customer onboarding using Amazon Connect _ NCS Case Study _ AWS.txt'
'Accelerating Migration at Scale Using AWS Application Migration Service with 3M Company _ Case Study _ AWS.txt'
'Accelerating Time to Market Using AWS and AWS Partner AccelByte _ Omeda Studios Case Study _ AWS.txt'
'Achieving Burstable Scalability and Consistent Uptime Using AWS Lambda with TiVo _ Case Study _ AWS.txt'
'Acrobits Uses Amazon Chime SDK to Easily Create Video Conferencing Application Boosting Collaboration for Global Users _ Acrobits Case Study _

### A criar os `chunk`s!

Listamos os documentos com a função que havíamos criado

In [14]:
dataset_path = "rag_txt_dataset"
documents = load_documents_from_directory(dataset_path)

Verificamos que o fizemos bem

In [15]:
for document in documents[0:10]:
    print(document["id"])

Run Jobs at Scale While Optimizing for Cost Using Amazon EC2 Spot Instances with ActionIQ _ ActionIQ Case Study _ AWS.txt
Recommend and dynamically filter items based on user context in Amazon Personalize _ AWS Machine Learning Blog.txt
Windsor.txt
Bank of Montreal Case Study _ AWS.txt
The Mill Adventure Case Study.txt
Optimize software development with Amazon CodeWhisperer _ AWS DevOps Blog.txt
Announcing enhanced table extractions with Amazon Textract _ AWS Machine Learning Blog.txt
THREAD _ Life Sciences _ AWS.txt
Deep Pool Optimizes Software Quality Control Using Amazon QuickSight _ Deep Pool Case Study _ AWS.txt
Upstox Saves 1 Million Annually Using Amazon S3 Storage Lens _ Upstox Case Study _ AWS.txt


Agora criamos os `chunk`s.

In [16]:
chunked_documents = []
for document in documents:
    chunks = split_text(document["text"])
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{document['id']}_{i}", "text": chunk})

In [17]:
len(chunked_documents)

3611

Como vemos, há 3611 `chunk`s. Como o limite diário da API da Hugging Face são 1000 chamadas na conta gratuita, se quisermos criar embeddings de todos os `chunk`s, acabaríamos as chamadas disponíveis e, além disso, não poderíamos criar embeddings de todos os `chunk`s

Lembramos que este modelo de embeddings é muito pequeno, somente 22M de parâmetros, por isso quase em qualquer computador pode ser executado, mais rápido ou mais lento, mas é possível.

Como só vamos a criar os embeddings dos `chunk`s uma vez, embora não tenhamos um computador muito potente e leve muito tempo, só será executado uma vez. Depois, quando quisermos fazer perguntas sobre a documentação, aí sim geraremos os embeddings do prompt com a API da Hugging Face e usaremos o LLM com a API. Portanto, só vamos ter que passar pelo processo de gerar os embeddings dos `chunk`s uma vez

Geramos os embeddings dos `chunk`s

Última biblioteca que vamos a ter que instalar. Como o processo de gerar os embeddings dos `chunk`s vai ser lento, vamos a instalar `tqdm` para que nos mostre uma barra de progresso. A instalamos com conda ou com pip, como preferir.

``` bash
conda install conda-forge::tqdm
```

o

``` bash
pip install tqdm
```

Geramos os embeddings dos `chunk`s

In [19]:
import tqdm

progress_bar = tqdm.tqdm(chunked_documents)

for chunk in progress_bar:
    embedding = get_embeddings(chunk["text"])
    if embedding is not None:
        chunk["embedding"] = embedding
    else:
        print(f"Error with document {chunk['id']}")

100%|██████████| 3611/3611 [00:16<00:00, 220.75it/s]


Vejamos um exemplo

In [60]:
from random import randint

idx = randint(0, len(chunked_documents))
print(f"Chunk id: {chunked_documents[idx]['id']},\n\ntext: {chunked_documents[idx]['text']},\n\nembedding shape: {chunked_documents[idx]['embedding'].shape}")

Chunk id: BNS Group Case Study _ Amazon Web Services.txt_0,

text: Reducing Virtual Machines from 40 to 12
The founders of BNS had been contemplating a migration from the company’s on-premises data center to the public cloud and observed a growing demand for cloud-based operations among current and potential BNS customers.
Français
Configures security according to cloud best practices
Clive Pereira, R&D director at BNS Group, explains, “The database that records Praisal’s SMS traffic resides in Praisal’s AWS environment. Praisal can now run complete analytics across its data and gain insights into what’s happening with its SMS traffic, which is a real game-changer for the organization.”  
Español
 AWS ISV Accelerate Program
 Receiving Strategic, Foundational Support from ISV Specialists
 Learn More
The value that AWS places on the ISV stream sealed the deal in our choice of cloud provider.” 
日本語
  Contact Sales 
BNS is an Australian software provider focused on secure enterprise SMS an

### Carregar os `chunk`s no banco de dados vetorial

Uma vez que temos todos os chunks gerados, os carregamos na base de dados vetorial. Voltamos a usar `tqdm` para que nos mostre uma barra de progresso, porque isso também vai ser lento

In [22]:
import tqdm

progress_bar = tqdm.tqdm(chunked_documents)

for chunk in progress_bar:
    collection.upsert(
        ids=[chunk["id"]],
        documents=chunk["text"],
        embeddings=chunk["embedding"],
    )

100%|██████████| 3611/3611 [00:59<00:00, 60.77it/s]


## Perguntas

Agora que temos a base de dados vetorial, podemos fazer perguntas à documentação. Para isso, precisamos de uma função que nos devolva o `chunk` correto

### Obter o `chunk` correto

Agora precisamos de uma função que nos devolva o `chunk` correto, vamos criá-la

In [50]:
def get_top_k_documents(query, k=5):
    results = collection.query(query_texts=query, n_results=k)
    return results

Por último, criamos uma `query`.

Para gerar a consulta, peguei aleatoriamente o documento `Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt`, passei-o para um LLM e pedi que gerasse uma pergunta sobre o documento. A pergunta que gerou é

```
Como a Neeva utilizou o Karpenter e as Instâncias Spot da Amazon EC2 para melhorar a gestão de infraestrutura e a otimização de custos?
```

Então obtemos os `chunk`s mais relevantes para essa pergunta

In [51]:
query = "How did Neeva use Karpenter and Amazon EC2 Spot Instances to improve its infrastructure management and cost optimization?"
top_chunks = get_top_k_documents(query=query, k=5)

Vamos a ver qué `chunk`s nos devolveu

In [52]:
for i in range(len(top_chunks["ids"][0])):
    print(f"Rank {i+1}: {top_chunks['ids'][0][i]}, distance: {top_chunks['distances'][0][i]}")

Rank 1: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_0, distance: 0.29233667254447937
Rank 2: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_5, distance: 0.4007825255393982
Rank 3: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_1, distance: 0.4317566752433777
Rank 4: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_6, distance: 0.43832334876060486
Rank 5: Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt_4, distance: 0.44625571370124817


Como eu havia dito, o documento que eu havia escolhido ao acaso era `Using Amazon EC2 Spot Instances and Karpenter to Simplify and Optimize Kubernetes Infrastructure _ Neeva Case Study _ AWS.txt` e como se pode ver os `chunk`s que nos devolveu são desse documento. Ou seja, de mais de 3000 `chunk`s que havia no banco de dados, foi capaz de devolver os `chunk`s mais relevantes diante disso, parece que isso funciona!

### Gerar a resposta

Como já temos os `chunk`s mais relevantes, passamos para o LLM, juntamente com a pergunta, para que este gere uma resposta

In [58]:
def generate_response(query, relevant_chunks, temperature=0.5, max_tokens=1024, top_p=0.7, stream=False):
    context = "\n\n".join([chunk for chunk in relevant_chunks])
    prompt = f"You are an assistant for question-answering. You have to answer the following question:\n\n{query}\n\nAnswer the question with the following information:\n\n{context}"
    message = [
        { "role": "user", "content": prompt }
    ]
    stream = client.chat.completions.create(
        messages=message, 
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stream=stream,
    )
    response = stream.choices[0].message.content
    return response

Testamos a função

In [59]:
response = generate_response(query, top_chunks["documents"][0])
print(response)

Neeva, a cloud-native, ad-free search engine founded in 2019, has leveraged Karpenter and Amazon EC2 Spot Instances to significantly improve its infrastructure management and cost optimization. Here’s how:

### Early Collaboration with Karpenter
In late 2021, Neeva began working closely with the Karpenter team, experimenting with and contributing fixes to an early version of Karpenter. This collaboration allowed Neeva to integrate Karpenter with its Kubernetes dashboard, enabling the company to gather valuable metrics on usage and performance.

### Combining Spot Instances and On-Demand Instances
Neeva runs its jobs on a large scale, which can lead to significant costs. To manage these costs effectively, the company adopted a combination of Amazon EC2 Spot Instances and On-Demand Instances. Spot Instances allow Neeva to bid on unused EC2 capacity, often at a fraction of the On-Demand price, while On-Demand Instances provide the necessary reliability for critical pipelines.

### Flexibi

Quando pedi ao LLM que me gerasse uma pergunta sobre o documento, também pedi que me gerasse a resposta correta. Esta é a resposta que o LLM me deu

``` text
A Neeva utilizou o Karpenter e as Instâncias Spot da Amazon EC2 para melhorar o gerenciamento de infraestrutura e a otimização de custos de várias maneiras:

Gerenciamento de Instâncias Simplificado:

Karpenter: Ao adotar o Karpenter, a Neeva simplificou o processo de provisionamento e gerenciamento de recursos de computação para seus clusters Amazon EKS. O Karpenter provisiona e desprovisiona instâncias automaticamente com base na carga de trabalho, eliminando a necessidade de configurações manuais e reduzindo a complexidade de entender diferentes instâncias de computação.
Instâncias Spot: A Neeva aproveitou as Instâncias Spot da Amazon EC2, que são capacidades EC2 não utilizadas disponíveis com um desconto significativo (até 90% de economia de custos). Isso permitiu que a empresa controlasse os custos ao mesmo tempo em que atendia aos seus requisitos de desempenho.
Escalabilidade Aprimorada:

Karpenter: A capacidade do Karpenter de dimensionar dinamicamente os recursos permitiu que a Neeva iniciasse novas instâncias rapidamente, permitindo que a empresa iterasse com uma velocidade maior e executasse mais experimentos em menos tempo.
Instâncias Spot: O uso de Instâncias Spot forneceu flexibilidade e diversificação de instâncias, tornando mais fácil para a Neeva dimensionar seus recursos de computação de forma eficiente.
Melhoria da Produtividade:

Karpenter: Ao democratizar as alterações de infraestrutura, o Karpenter permitiu que qualquer engenheiro modificasse as configurações do Kubernetes, reduzindo a dependência de especialidades. Isso economizou para a equipe da Neeva até 100 horas por semana de tempo de espera na administração de sistemas.
Instâncias Spot: A capacidade de provisionar e desprovisionar rapidamente instâncias Spot reduziu os atrasos no pipeline de desenvolvimento, garantindo que os trabalhos não ficassem presos devido à falta de recursos disponíveis.
Eficiência de Custo:

Karpenter: As melhores práticas do Karpenter para Instâncias Spot, incluindo flexibilidade e diversificação de instâncias, ajudaram a Neeva a usar essas instâncias de forma mais eficaz, permanecendo dentro do orçamento.
Instâncias Spot: As economias de custo decorrentes do uso de Instâncias Spot permitiram que a Neeva executasse trabalhos em larga escala, como indexação, por quase o mesmo custo, mas em uma fração do tempo. Por exemplo, a Neeva reduziu seus trabalhos de indexação de 18 horas para apenas 3 horas.
Melhor Utilização de Recursos:

Karpenter: O Karpenter forneceu uma visibilidade melhor sobre o uso de recursos de computação, permitindo que a Neeva acompanhasse e otimizasse seu consumo de recursos de forma mais próxima.
Instâncias Spot: A combinação de Karpenter e Instâncias Spot permitiu que a Neeva executasse grandes modelos de linguagem de forma mais eficiente, aprimorando a experiência de pesquisa para seus usuários.
Em resumo, a adoção do Karpenter e das Instâncias Spot da Amazon EC2 pela Neeva melhorou significativamente a gestão de infraestrutura, a otimização de custos e a eficiência geral de desenvolvimento, permitindo que a empresa forneça melhores experiências de busca sem anúncios aos seus usuários.
```

E esta foi a resposta gerada pelo nosso `RAG`

``` text
Neeva, um mecanismo de busca nativo em nuvem e sem anúncios, fundado em 2019, aproveitou o Karpenter e as Instâncias Spot da Amazon EC2 para melhorar significativamente a gestão de sua infraestrutura e a otimização de custos. Veja como:

### Colaboração Inicial com Karpenter
No es necesario traducir a portugues, el texto dice que se debe traducir al portugues texto markdown, pero el texto proporcionado ya está en español, sin embargo, si se refiere a traducir al portugues, aquí está la traducción:

Fim de 2021, a Neeva começou a trabalhar em estreita colaboração com a equipe do Karpenter, experimentando e contribuindo com correções para uma versão inicial do Karpenter. Essa colaboração permitiu que a Neeva integrasse o Karpenter com o painel do Kubernetes, habilitando a empresa a coletar métricas valiosas sobre uso e desempenho.

### Combinando Instâncias Spot e Instâncias On-Demand
A Neeva executa seus trabalhos em larga escala, o que pode levar a custos significativos. Para gerenciar esses custos de forma eficaz, a empresa adotou uma combinação de Instâncias Spot da Amazon EC2 e Instâncias On-Demand. As Instâncias Spot permitem que a Neeva faça ofertas sobre a capacidade não utilizada da EC2, frequentemente por uma fração do preço On-Demand, enquanto as Instâncias On-Demand fornecem a confiabilidade necessária para pipelines críticos.

### Flexibilidade e Diversificação de Instâncias
De acordo com Mohit Agarwal, líder de engenharia de infraestrutura da Neeva, a adoção de Karpenter das melhores práticas para Instâncias Spot, incluindo flexibilidade e diversificação de instâncias, foi crucial. Essa abordagem garante que a Neeva possa ajustar dinamicamente seus recursos de computação para atender a cargas de trabalho variadas, minimizando os custos.

### Melhorada Escalabilidade e Agilidade
Ao utilizar o Karpenter para provisionar recursos de infraestrutura para seus clusters Amazon EKS, a Neeva alcançou vários benefícios importantes:
- **Escalabilidade**: Neeva pode dimensionar seus recursos de computação para cima ou para baixo conforme necessário, garantindo que sempre tenha a capacidade necessária para lidar com suas cargas de trabalho.
- **Agilidade**: A empresa pode iterar rapidamente e democratizar as alterações de infraestrutura, reduzindo o tempo gasto em administração de sistemas em até 100 horas por semana.

### Ciclos de Desenvolvimento Aprimorados
A integração do Karpenter e das Instâncias Spot também acelerou os ciclos de desenvolvimento da Neeva. A empresa agora pode lançar novos recursos e melhorias mais rapidamente, o que é essencial para manter uma vantagem competitiva no mercado de mecanismos de busca.

### Economia de Custo e Controle Orçamentário
Usando Instâncias Spot, a Neeva conseguiu permanecer dentro do seu orçamento ao mesmo tempo em que atende aos seus requisitos de desempenho. Essa otimização de custos é fundamental para uma empresa que prioriza experiências centradas no usuário e não tem incentivos concorrentes de publicidade.

### Planos Futuros
A Neeva está comprometida em continuar sua inovação e expansão. A empresa planeja lançar em novas regiões e melhorar ainda mais seu mecanismo de busca, mantendo ao mesmo tempo a eficiência de custos. Como Mohit Agarwal observa, "A maior parte de nosso processamento é ou será gerenciada usando Karpenter em frente."

### Conclusão
Aproveitando Karpenter e Instâncias Spot da Amazon EC2, a Neeva não apenas otimizou seus custos de infraestrutura, mas também melhorou sua escalabilidade, agilidade e velocidade de desenvolvimento. Essa abordagem estratégica posicionou a Neeva para oferecer experiências de busca de alta qualidade e sem anúncios aos seus usuários, mantendo um forte foco no controle de custos e inovação.
```

Por lo que podemos concluir que el `RAG` ha funcionado correctamente!!! 

Debería ser:
 
Por lo que podemos concluir que o `RAG` funcionou corretamente!!!